In [26]:
import json
from pathlib import Path
import pandas as pd
import os
import pprint
class Error:
    def __init__(self, assetId, field, expectedValueRange, actualValue, criticality, timestamp):
        self.assetId = assetId
        self.field = field
        self.expectedValueRange = expectedValueRange
        self.actualValue = actualValue
        self.criticality = criticality
        self.timestamp = timestamp
    def __str__(self):
        return "Test failled for asset id " + str(self.assetId) + ". Expected value for field " + self.field + " was in range " + str(self.expectedValueRange) + ", received value " + str(self.actualValue) + " at " + self.timestamp
        
class Detectors:
    def __init__(self):
        self.missions = []
        self.sites = []
        self.assets = []
        self.load_config()
        self.load_assets()
        self.connect_to_carbon()
        
    def load_config(self):
        data = None
        with open('./config.json') as f:
            data = json.load(f)
        self.missions = data["missions"]
        self.sites = data["sites"]
        
    def load_assets(self):        
        p = Path('./assets')
        for child in p.iterdir():
            filename, ext = os.path.splitext(str(child))
            if(ext == ".json"):
                with open(str(child)) as f:
                    self.assets.append(json.load(f))
                    
    def connect_to_carbon(self):
        self.carbon = {}
    
    def within(self, val, v_range):
        return val >= v_range[0] and val <= v_range[1]
                    
    def getLatestData(self, id):
        df = pd.read_csv("../mocker/"+str(id)+".csv")
        return df.iloc[-1]
    
    def runTests(self):
        errors = []
        for asset in self.assets:
            data = self.getLatestData(asset["id"])
#             tests = []
#             if(hasattr(asset,"tests_preset")):
#                 with open('./assets/presets/'+asset["tests_preset"]+'.json') as f:
#                     tests = tests + json.load(f)["tests"]
                
#                 tests = tests + asset["tests"]
#             print(tests)

            for test in asset["tests"]:
                # IF conditions must be met for a test to be accurate
                if len(test["if"]) > 0:
                    conditions_met = True
                    for condition in test["if"]:
                        if not self.within(data[condition["field"]],condition["value_range"]):
                            conditions_met = False
                    
                    if conditions_met:
                        for condition in test["then_conditions"]:
                            print(data[condition["field"]])
                            if not self.within(data[condition["field"]], condition["value_range"]):
                                errors.append(Error(asset["id"], condition["field"], condition["value_range"], data[condition["field"]], test["test_classification"], data["datetime"]))
                            
                    else:
                        # If the conditions are not met, the test is not run
                        0
                
                # If there are no prerequisits to running the test
                else:
                    for condition in test["then_conditions"]:
                        if not self.within(data[condition["field"]], condition["value_range"]):
                            errors.append(Error(asset["id"], condition["field"], condition["value_range"], data[condition["field"]], test["test_classification"], data["datetime"]))
        return errors
    
    def print(self):
        pp = pprint.PrettyPrinter(indent=4)
        print("\n--- Missions ---")
        pp.pprint(self.missions)
        print("\n--- Sites ---")
        pp.pprint(self.sites)
        print("\n--- Assets ---")
        pp.pprint(self.assets)
        

a = Detectors()
errors = a.runTests()
if (len(errors) == 0):
    print("All tests passed")
for e in errors:
    print(e)

Test failled for asset id 0. Expected value for field data1 was in range [200, 240], received value 191.873171142148 at 2016-01-01 06:00:00


#### Once we have the errors, this is probably when we want to call carbon

In [44]:
class CriticalityTool:
    def __init__(self):
        0
    def getCriticality(self, id):
        p = Path('./assets')
        for child in p.iterdir():
            filename, ext = os.path.splitext(str(child))
            if(ext == ".json"):
                with open(str(child)) as f:
                    asset = json.load(f)
                    if(asset["id"] == id):
                        return {"name": asset["name"], "site": asset["site_id"], "missions": asset["mission_ids"], "criticalities":asset["mission_criticalities"]}
#                     THIS FUNCTION DOES NOT HANDLE ERRORS CURRENTLY
    def changeCriticality(self, id, mission_id, newCrit):
        p = Path('./assets')
        for child in p.iterdir():
            filename, ext = os.path.splitext(str(child))
            if(ext == ".json"):
                with open(str(child)) as f:
                    asset = json.load(f)
                    if(asset["id"] == id):
                        for i in range(len(asset["mission_ids"])):
                            if asset["mission_ids"][i] == mission_id:
                                asset["mission_criticalities"][i] = newCrit
                                with open(child, 'w') as outfile:
                                    json.dump(asset, outfile)
                                return 1
        print("fail")

dc = CriticalityTool()
dc.changeCriticality(0, 0, 4)
dc.getCriticality(0)

{'name': 'Raytheon XM100 Air Threat Radar',
 'site': 0,
 'missions': [0],
 'criticalities': [4]}

# TODO
- Need to pull in actual data
- Need to account for error cases where data or fields do not exist
- Integrate carbon api